In [ ]:
#!pip install GitPython
#!pip install sagemaker
#!pip install s3fs
#!pip install aws-mlops
import os
import sys
import pandas as pd
import config as config
#sys.path.insert(1, os.getcwd() + '/..')
from aws_mlops.data_storage import DataStorage
from sklearn.model_selection import train_test_split

In [ ]:
# object for data storage management
studio = DataStorage(config.source_bucket, config.test_data_key)
config_name = 'pretraining_input'

In [ ]:
# load dataframe after loading on s3
df = studio.restore(config.raw_data_filename, f's3://{config.source_bucket}/{config.raw_data_key}')

In [ ]:
# --> add your code for first split
df.insert(0, 'id', range(883, 883 + len(df)))

In [ ]:
# prep data
train, test, _, _ = train_test_split(df, df[config.target], test_size=config.test_size, random_state=0)

In [ ]:
# checkpoint / restore - sample
studio.checkpoint(df)
df = studio.restore()

In [ ]:
# --> add your code for second split
train.drop('id', axis = 1, inplace = True)

In [ ]:
# prep data
final_train, validation, _, _ = train_test_split(train, train[config.target], test_size=config.validation_size, random_state=0)

In [ ]:
# output - columns names, target and identifier columns
test = studio.save_test(test, [config.target, config.identifier])

In [ ]:
# output - dataframes: train, validation, test
output_dataframes = [final_train, validation, test]
for output in getattr(config, config_name)['outputs']:
    dataframe = output_dataframes.pop(0)
    studio.save_on_s3(df, output['OutputName'], output['S3Output']['S3Uri'], header = False, index = False)
    #studio.local_save(dataframe, output['S3Output']['LocalPath'], output['OutputName'], header = False, index = False)